# 🔗 Meta-Agent Factory System - LangChain Version

## 📋 Overview
This notebook demonstrates an alternative implementation of the Meta-Agent Factory System using **LangChain** and **LangGraph** frameworks with **OpenAI GPT-4o**.

### 🎯 What This System Does
- **Automated Agent Creation**: Generates complete AI agent systems from natural language descriptions
- **Human-in-the-Loop Design**: Interactive verification at each stage (strategy selection, blueprint approval)
- **Multi-Agent Architecture**: Three specialized agents collaborate to design and build your solution

### 🔄 Comparison with Google ADK Version
| Feature | **LangChain Version** | **Google ADK Version** |
|---------|----------------------|------------------------|
| Framework | LangChain + LangGraph | Google Agent Dev Kit |
| LLM Backend | OpenAI GPT-4o | Google Gemini |
| State Management | TypedDict + Graph | Native ADK State |
| Orchestration | StateGraph | Custom Loop |
| Tool Support | LangChain Tools | ADK Tools |

### 🏗️ System Architecture
```
┌─────────────────────────────────────────────────────────────┐
│                    META-AGENT FACTORY                        │
│                    (LangChain Version)                       │
└─────────────────────────────────────────────────────────────┘
                            │
                            ▼
        ┌───────────────────────────────────────┐
        │  PHASE 1: STRATEGY CONSULTATION       │
        │  Agent: Consultant (GPT-4o)          │
        │  Output: 2 Architecture Options      │
        └───────────────────────────────────────┘
                            │
                    [User Chooses Option]
                            │
                            ▼
        ┌───────────────────────────────────────┐
        │  PHASE 2: BLUEPRINT DESIGN            │
        │  Agent: Architect (GPT-4o)           │
        │  Output: JSON System Blueprint       │
        └───────────────────────────────────────┘
                            │
                    [Revision Loop]
                    /              \
          User Approves?     User Requests Changes
                  │                   │
                  │              [Back to Architect]
                  ▼
        ┌───────────────────────────────────────┐
        │  PHASE 3: CODE GENERATION             │
        │  Agent: Builder (GPT-4o)             │
        │  Output: Executable Python Code      │
        └───────────────────────────────────────┘
                            │
                            ▼
                  [Save to File]
```

### 🚀 Key Differences from ADK Version
1. **Explicit Graph Structure**: Uses LangGraph's StateGraph for flow control
2. **OpenAI Models**: Leverages GPT-4o instead of Google Gemini
3. **LangChain Ecosystem**: Access to extensive LangChain tool library
4. **Manual Flow Control**: Developer manages execution flow between human inputs

---

In [ ]:
# ============================================================================
# ENVIRONMENT SETUP & API KEY LOADING
# ============================================================================
# This cell loads environment variables and validates API key availability.
# It uses override=True to force reload, preventing Jupyter kernel caching issues.
# ============================================================================

from dotenv import load_dotenv
import os

# Force reload .env (override=True ensures fresh values from .env)
# This is critical in Jupyter where environment persists across executions
load_dotenv(override=True)

# Validate API key and provide diagnostic info
# We show partial key (first 7 chars) for verification without exposing full key
openai_key = os.getenv("OPENAI_API_KEY")
if openai_key:
    print(f"✅ API Key loaded successfully (starts with: {openai_key[:7]}...)")
else:
    print("❌ API Key NOT found in .env")
    print("📝 Please add OPENAI_API_KEY=your_key to .env file")
    print("🔄 After adding, restart kernel (Kernel → Restart) and re-run this cell")

## 🔐 Step 1: Environment Setup & API Key Validation

### What This Cell Does
- Loads environment variables from `.env` file
- Validates that `OPENAI_API_KEY` is present
- Uses `override=True` to ensure fresh values (important in Jupyter notebooks)

### 🔑 Required Environment Variables
Create a `.env` file in the project root with:
```env
OPENAI_API_KEY=sk-proj-your_actual_key_here
```

### 🛡️ Security Best Practices
- ✅ **Never hardcode API keys** in notebooks or source code
- ✅ **Add `.env` to `.gitignore`** to prevent accidental commits
- ✅ **Use environment variables** for all sensitive credentials
- ✅ **Rotate keys regularly** and use scoped permissions

### 📌 Note
If you see "❌ API Key NOT found", ensure:
1. `.env` file exists in project root
2. File contains `OPENAI_API_KEY=your_key`
3. Kernel has been restarted (Kernel → Restart) after editing `.env`

In [ ]:
# ============================================================================
# LLM MODEL INITIALIZATION
# ============================================================================
# Initialize the Large Language Model (LLM) that powers all three agents.
# We use GPT-4o with temperature=0 for deterministic, reproducible outputs.
# ============================================================================

import os
from typing import TypedDict, Literal
from dotenv import load_dotenv

# Load environment variables (redundant check for safety)
load_dotenv()

# --- 1. SETUP THE MODEL ---
# ChatOpenAI: LangChain wrapper for OpenAI's chat models
# Alternatives: ChatAnthropic (Claude), ChatGoogleGenerativeAI (Gemini)
from langchain_openai import ChatOpenAI

# Critical validation: Ensure API key is available before initializing model
if "OPENAI_API_KEY" not in os.environ:
    raise RuntimeError(
        "OPENAI_API_KEY not set. "
        "Add it to your .env file and restart the kernel."
    )

# Initialize the LLM with optimal settings for agent factory
# - model="gpt-4o": Latest GPT-4 Optimized for complex reasoning
# - temperature=0: Deterministic outputs (crucial for consistent blueprints)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

print("✅ Model Initialized (GPT-4o, Temperature=0)")
print("🎯 Ready for multi-agent orchestration")

## 🤖 Step 2: Model Initialization

### What This Cell Does
- Initializes the **ChatOpenAI** LLM with GPT-4o model
- Configures model with `temperature=0` for deterministic outputs
- Validates that API key is properly configured

### 🎯 Model Configuration
| Parameter | Value | Purpose |
|-----------|-------|---------|
| **Model** | `gpt-4o` | Latest GPT-4 Optimized model for complex reasoning |
| **Temperature** | `0` | Deterministic outputs (no randomness) for consistent blueprints |
| **Provider** | OpenAI | Required for LangChain ChatOpenAI integration |

### 🔄 Alternative Models
You can swap to other providers:
```python
# Anthropic Claude
from langchain_anthropic import ChatAnthropic
llm = ChatAnthropic(model="claude-3-sonnet-20240229")

# Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro")
```

### ⚡ Performance Considerations
- **GPT-4o**: Best for complex reasoning, architectural design
- **GPT-3.5-turbo**: Faster and cheaper, good for simpler tasks
- **Temperature=0**: Ensures reproducible blueprints and code generation

## 📊 Step 3: Define the State (The Memory)

### What is State in LangGraph?
State is a **shared memory structure** that passes between agents, accumulating information as the workflow progresses.

### 🧠 State Structure
```python
FactoryState = {
    "user_initial_request": str,      # Original problem description
    "consultant_options": str,        # 2 architecture strategies from Consultant
    "user_strategy_choice": str,      # User's selected strategy (Option 1 or 2)
    "current_blueprint": str,         # JSON blueprint from Architect
    "user_feedback": str,             # User's revision requests
    "final_code": str,                # Generated Python code from Builder
    "status": str                     # Workflow state tracking
}
```

### 🔄 Status Field Values
| Status | Meaning | Next Action |
|--------|---------|-------------|
| `planning` | Consultant generating options | Wait for user choice |
| `waiting_for_user_strategy` | Options presented | User selects strategy |
| `reviewing` | Architect designing blueprint | Wait for user approval |
| `waiting_for_approval` | Blueprint ready | User approves/revises |
| `revision_requested` | User wants changes | Back to Architect |
| `approved` | Blueprint accepted | Proceed to Builder |
| `coding` | Builder generating code | Final step |
| `finished` | Code generation complete | Save to file |

### 🎯 Why TypedDict?
- **Type Safety**: Ensures all agents use correct field names
- **IDE Support**: Autocomplete and type hints in development
- **Documentation**: Self-documenting state structure

In [ ]:
# ============================================================================
# STATE DEFINITION - THE SHARED MEMORY
# ============================================================================
# FactoryState is a TypedDict that acts as shared memory across all agents.
# Each agent reads from and writes to this state as the workflow progresses.
# This replaces the manual state tracking in the Google ADK version.
# ============================================================================

class FactoryState(TypedDict):
    """
    Shared state object that flows through the agent pipeline.
    
    Fields:
        user_initial_request: Original problem description from user
        consultant_options: Strategy proposals (Simple vs Complex)
        user_strategy_choice: User's selected strategy ("Option 1" or "Option 2")
        current_blueprint: JSON architecture blueprint from Architect
        user_feedback: User's revision requests for blueprint changes
        final_code: Complete Python code generated by Builder
        status: Current workflow stage (planning|reviewing|coding|finished)
    """
    user_initial_request: str     # e.g., "I want a YouTube script writer agent"
    consultant_options: str       # Consultant's 2 proposed approaches
    user_strategy_choice: str     # User selects "Option 1" or "Option 2"
    current_blueprint: str        # JSON blueprint from Architect
    user_feedback: str            # User's comments for revisions
    final_code: str               # Final generated Python code
    status: str                   # Tracks workflow state transitions

## 🤝 Step 4: Define the Agents (The Nodes)

### What are Nodes in LangGraph?
Nodes are **Python functions** that represent individual agents. Each node:
1. Takes the current `State` as input
2. Performs its specialized task (consulting, designing, coding)
3. Returns updates to the `State`

### 🎭 The Three Specialized Agents

#### 1. 👔 **Consultant Agent**
- **Role**: AI Solutions Consultant
- **Input**: User's initial request
- **Task**: Propose 2 architectural strategies (Simple vs Complex)
- **Output**: `consultant_options` (text description of both options)
- **Status**: Sets `status="waiting_for_user_strategy"`

#### 2. 📐 **Architect Agent**
- **Role**: Systems Architect
- **Input**: User's chosen strategy + any feedback from previous iteration
- **Task**: Design JSON blueprint with agents, roles, tools, and workflow
- **Output**: `current_blueprint` (JSON structure)
- **Special Feature**: Revision loop - incorporates user feedback
- **Status**: Sets `status="waiting_for_approval"`

#### 3. 👷 **Builder Agent**
- **Role**: Python Code Expert
- **Input**: Approved blueprint
- **Task**: Generate complete, executable Python code using LangChain/LangGraph
- **Output**: `final_code` (ready-to-run Python script)
- **Status**: Sets `status="finished"`

### 🔄 Node Execution Flow
```
User Request → Consultant → [User Choice] → Architect → [Approval Loop] → Builder → Code
```

### 🎯 Key Design Patterns
- **Single Responsibility**: Each agent has one clear purpose
- **State Updates**: Agents return dictionaries that merge into main state
- **Context Awareness**: Architect checks for feedback and adjusts accordingly
- **Clean Output**: Responses stripped of markdown formatting for parsing

In [ ]:
# ============================================================================
# AGENT DEFINITIONS - THE THREE SPECIALIZED NODES
# ============================================================================
# Each function below represents an autonomous agent with a specific role.
# They communicate through the shared FactoryState, updating it as they work.
# ============================================================================

from langchain_core.messages import SystemMessage, HumanMessage

# ============================================================================
# AGENT 1: THE CONSULTANT 👔
# ============================================================================
# Role: AI Solutions Consultant
# Responsibility: Analyze user request and propose 2 architectural strategies
# ============================================================================
def consultant_node(state: FactoryState):
    """
    Consultant Agent: Proposes two architectural approaches.
    
    Args:
        state: Current FactoryState with user_initial_request
        
    Returns:
        dict: Updated state with consultant_options and status
        
    Example Output:
        "Option 1 (Simple): Single agent with predefined script template...
         Option 2 (Complex): Multi-agent system with research, writing, editing..."
    """
    print("\n--- 👔 CONSULTANT IS THINKING ---")
    
    # Construct strategic analysis prompt
    prompt = f"""
    You are an AI Solutions Consultant. The user wants: "{state['user_initial_request']}".
    
    Your task: Propose 2 distinct architectural options for an AI Agent system to solve this.
    1. A Simple/Direct approach (faster, fewer components, good for MVP)
    2. A Complex/Comprehensive approach (robust, scalable, production-ready)
    
    Be concise but specific about the agent roles and workflow in each option.
    """
    
    # Invoke LLM for strategic recommendations
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "consultant_options": response.content,
        "status": "waiting_for_user_strategy"
    }


# ============================================================================
# AGENT 2: THE ARCHITECT 📐
# ============================================================================
# Role: Systems Architect
# Responsibility: Design detailed JSON blueprint based on chosen strategy
# Special Feature: Revision loop - incorporates user feedback
# ============================================================================
def architect_node(state: FactoryState):
    """
    Architect Agent: Designs the system blueprint in JSON format.
    
    Args:
        state: Current FactoryState with user_strategy_choice and optional user_feedback
        
    Returns:
        dict: Updated state with current_blueprint (JSON) and status
        
    Revision Loop:
        If user_feedback exists, the prompt includes it to fix the design.
        This allows iterative refinement until user approves.
        
    Example Blueprint:
        {
          "system_name": "YouTube Script Writer",
          "agents": [
            {"name": "Researcher", "role": "Topic research", "tools": ["web_search"]},
            {"name": "Writer", "role": "Script creation", "tools": ["llm_generate"]}
          ],
          "flow": "Sequential"
        }
    """
    print("\n--- 📐 ARCHITECT IS DESIGNING ---")
    
    # Check if this is a revision (user provided feedback on previous blueprint)
    feedback_context = ""
    if state.get("user_feedback"):
        feedback_context = f"""
        USER FEEDBACK ON PREVIOUS DRAFT: {state['user_feedback']}
        Fix the design accordingly. Address all concerns mentioned above.
        """

    # Construct architecture design prompt with context
    prompt = f"""
    You are a Systems Architect specializing in AI agent systems.
    
    User Goal: {state['user_initial_request']}
    Selected Strategy: {state['user_strategy_choice']}
    {feedback_context}

    Design a JSON blueprint for this agent system with the following structure:
    {{
      "system_name": "descriptive_name",
      "agents": [
        {{
          "name": "AgentName",
          "role": "What this agent does",
          "tools": ["tool1", "tool2"]
        }}
      ],
      "flow": "Sequential" or "Parallel" or "Conditional"
    }}
    
    Output ONLY the JSON. No explanations, no markdown formatting.
    """
    
    # Invoke LLM for blueprint generation
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Clean up any markdown code fencing that might be included
    blueprint = response.content.replace("```json", "").replace("```", "").strip()
    
    return {
        "current_blueprint": blueprint,
        "status": "waiting_for_approval"
    }


# ============================================================================
# AGENT 3: THE BUILDER 👷
# ============================================================================
# Role: Python Code Expert
# Responsibility: Generate complete, executable Python code from blueprint
# ============================================================================
def builder_node(state: FactoryState):
    """
    Builder Agent: Generates production-ready Python code.
    
    Args:
        state: Current FactoryState with approved current_blueprint
        
    Returns:
        dict: Updated state with final_code and status="finished"
        
    Code Generation:
        - Uses LangChain and LangGraph frameworks
        - Creates StateGraph with proper nodes and edges
        - Includes all imports and configuration
        - Ready to save and execute
        
    Example Output:
        Complete Python script with:
        - Imports (langchain_openai, langgraph)
        - State definition
        - Agent functions
        - Graph construction
        - Execution logic
    """
    print("\n--- 👷 BUILDER IS CODING ---")
    
    # Construct code generation prompt with detailed requirements
    prompt = f"""
    You are a Python Expert specializing in LangChain and LangGraph.
    
    Write the complete, executable code for this AI agent system:
    
    Blueprint:
    {state['current_blueprint']}
    
    Requirements:
    - Use `langchain_openai` for LLM integration
    - Use `langgraph` for workflow orchestration
    - Create a StateGraph with proper state management
    - Define nodes for each agent
    - Add edges to connect the workflow
    - Compile the graph
    - Include proper error handling
    - Add comments for clarity
    
    Output ONLY valid Python code. No explanations, no markdown formatting.
    Make it production-ready and well-structured.
    """
    
    # Invoke LLM for code generation
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Clean up markdown code fencing
    code = response.content.replace("```python", "").replace("```", "").strip()
    
    return {
        "final_code": code,
        "status": "finished"
    }

## 🕸️ Step 5: Build the Graph (The Orchestration Logic)

### What is a StateGraph?
A **StateGraph** is LangGraph's way of defining agent workflows. It replaces the manual `while` loops from the Google ADK version with a declarative graph structure.

### 🏗️ Graph Components

#### 1. **Nodes** (The Agents)
- `consultant`: Proposes strategies
- `architect`: Designs blueprints
- `builder`: Generates code

#### 2. **Edges** (The Connections)
- Define how agents connect
- Control flow between nodes
- Can be conditional or fixed

#### 3. **Router Function** (The Decision Maker)
- Determines which node runs next
- Based on current state
- Handles human-in-the-loop pauses

### 🔄 Flow Control Logic
```
START → Consultant → END (pause for user choice)
      → Architect → END (pause for approval)
      → Builder → END (complete)
```

### 🎯 Key Design Decisions

#### Why END Instead of Direct Edges?
We use `END` after each agent because:
- **Human Input Required**: User must choose strategy, approve blueprint
- **Interactive Workflow**: Cannot be fully automated
- **Manual Orchestration**: We control execution from outside the graph

#### Alternative Approach
For fully automated workflows, you could use:
```python
workflow.add_conditional_edges("consultant", router_function)
```

### 🚀 Graph Compilation
`app = workflow.compile()` creates an executable workflow that:
- Validates the graph structure
- Optimizes execution paths
- Provides streaming and invoke methods

In [ ]:
# ============================================================================
# GRAPH CONSTRUCTION - THE WORKFLOW ORCHESTRATOR
# ============================================================================
# This cell defines the agent workflow using LangGraph's StateGraph.
# Unlike the Google ADK version's manual loop, this creates a declarative
# graph structure that manages agent execution flow.
# ============================================================================

from langgraph.graph import StateGraph, END

# ============================================================================
# STEP 1: Initialize the Graph
# ============================================================================
# StateGraph manages the workflow and state transitions between agents
workflow = StateGraph(FactoryState)

# ============================================================================
# STEP 2: Add Agent Nodes
# ============================================================================
# Each node is a function that processes state and returns updates
workflow.add_node("consultant", consultant_node)  # Strategy proposals
workflow.add_node("architect", architect_node)    # Blueprint design
workflow.add_node("builder", builder_node)        # Code generation

# ============================================================================
# STEP 3: Define the Router (Conditional Logic)
# ============================================================================
# Router determines which node to execute next based on current state.
# This replaces the while-loop logic from the Google ADK version.
# ============================================================================
def router(state: FactoryState):
    """
    Flow control logic that determines the next node to execute.
    
    Returns:
        str: Name of next node ("consultant"|"architect"|"builder") or END
        
    Flow Decision Tree:
        - No options yet? → "consultant"
        - Options presented, awaiting choice? → END (pause for human)
        - Choice made, no blueprint? → "architect"
        - Blueprint ready, awaiting approval? → END (pause for human)
        - Blueprint approved? → "builder"
        - Revision requested? → "architect" (loop back)
        - Everything done? → END (complete)
    """
    # Starting point: Run consultant if we don't have options yet
    if not state.get("consultant_options"):
        return "consultant"
    
    # Pause for user to choose strategy
    if state["status"] == "waiting_for_user_strategy":
        return END 
        
    # Run architect if we haven't generated a blueprint yet
    if not state.get("current_blueprint"):
        return "architect"
        
    # Pause for user to approve/reject blueprint
    if state["status"] == "waiting_for_approval":
        return END
    
    # User approved: proceed to code generation
    if state["status"] == "approved":
        return "builder"
    
    # User requested changes: loop back to architect with feedback
    if state["status"] == "revision_requested":
        return "architect"
        
    # Workflow complete
    return END

# ============================================================================
# STEP 4: Configure Graph Edges
# ============================================================================
# Set entry point (where the graph starts)
workflow.set_entry_point("consultant")

# Add edges to END after each node
# We use END because we need human input between agents
# For fully automated flows, you'd use: workflow.add_conditional_edges("node", router)
workflow.add_edge("consultant", END)  # Pause for strategy selection
workflow.add_edge("architect", END)   # Pause for blueprint approval
workflow.add_edge("builder", END)     # Complete after code generation

# ============================================================================
# STEP 5: Compile the Graph
# ============================================================================
# Compilation validates structure and creates executable workflow
app = workflow.compile()

print("✅ Factory Graph Compiled Successfully")
print("📊 Graph Structure:")
print("   Nodes: consultant → architect → builder")
print("   Flow: Human-in-the-loop at each stage")
print("   Ready for execution!")

## 🎮 Step 6: The Interactive Execution Loop

### What Happens Here?
This cell orchestrates the **entire agent factory workflow** with human checkpoints at each stage.

### 🔄 Execution Phases

#### **Phase 1: Consultation** 👔
1. Initialize state with user's request
2. Run Consultant agent via `app.stream(state)`
3. Display 2 proposed strategies
4. **[PAUSE]** Wait for user to choose Option 1 or Option 2

#### **Phase 2: Blueprint Design** 📐
1. Update state with user's strategy choice
2. Run Architect agent via `app.invoke(state)`
3. Display JSON blueprint
4. **[VERIFICATION LOOP]**:
   - User types "Approved" → Proceed to Phase 3
   - User provides feedback → Architect revises blueprint
   - Loop continues until approval

#### **Phase 3: Code Generation** 👷
1. Set status to "approved"
2. Run Builder agent
3. Generate complete Python code
4. Display and save to file

### 🎯 Key Concepts

#### Why Manual Execution?
Unlike fully automated agents, this system requires **human judgment** at critical decision points:
- **Strategy Selection**: Business decision (Simple vs Complex)
- **Blueprint Approval**: Technical review before implementation
- **Iterative Refinement**: Feedback loop for perfect design

#### State Management
```python
state.update(output)  # Merge agent updates into main state
```

#### Graph Execution Methods
- **`app.stream(state)`**: Streaming execution, get updates as they happen
- **`app.invoke(state)`**: Synchronous execution, wait for completion

### 📁 Output
Generated code saves to: `generated_langchain_agent.py`

### 🚀 Usage Pattern
```
Run Cell → Enter Request → Choose Strategy → Review Blueprint → 
Approve/Revise → Get Code → Done!
```

In [ ]:
# ============================================================================
# INTERACTIVE EXECUTION - THE MAIN ORCHESTRATION LOOP
# ============================================================================
# This cell runs the complete agent factory workflow with human checkpoints.
# It demonstrates human-in-the-loop design: AI proposes, human decides.
# ============================================================================

# ============================================================================
# INITIALIZATION - Set Up the Starting State
# ============================================================================
# Define what agent system you want to create
# Modify this request to generate different types of agent systems
state = {
    "user_initial_request": "I want a YouTube script writer agent."
}

print("=" * 70)
print("🚀 META-AGENT FACTORY - LANGCHAIN VERSION")
print("=" * 70)
print(f"🎯 GOAL: {state['user_initial_request']}")
print("=" * 70)

# ============================================================================
# PHASE 1: STRATEGY CONSULTATION
# ============================================================================
# Run the Consultant agent to get architectural options
print("\n" + "=" * 70)
print("PHASE 1: STRATEGY CONSULTATION")
print("=" * 70)

# Stream execution allows us to see updates as they happen
for output in app.stream(state):
    # output is a dict with the node name as key
    if 'consultant' in output:
        # Merge consultant's updates into our state
        state.update(output['consultant'])

# Display the consultant's recommendations
print(f"\n📋 CONSULTANT'S RECOMMENDATIONS:")
print("-" * 70)
print(state['consultant_options'])
print("-" * 70)

# ============================================================================
# HUMAN CHECKPOINT 1: Strategy Selection
# ============================================================================
# User must choose between Simple and Complex approaches
choice = input("\n👤 Choose your strategy (e.g., 'Option 1' or 'Option 2'): ")
state["user_strategy_choice"] = choice
print(f"✅ Selected: {choice}")

# ============================================================================
# PHASE 2: BLUEPRINT DESIGN (WITH REVISION LOOP)
# ============================================================================
print("\n" + "=" * 70)
print("PHASE 2: ARCHITECTURAL DESIGN")
print("=" * 70)

# Run architect for initial blueprint
output = app.invoke(state, config={"configurable": {"thread_id": "1"}})
state.update(output)

# ============================================================================
# THE VERIFICATION LOOP - Iterative Blueprint Refinement
# ============================================================================
# This loop continues until user approves the blueprint
# User can request changes, and Architect revises accordingly
iteration = 1
while True:
    print(f"\n📐 BLUEPRINT PROPOSAL (Iteration {iteration}):")
    print("-" * 70)
    print(state['current_blueprint'])
    print("-" * 70)
    
    # ============================================================================
    # HUMAN CHECKPOINT 2: Blueprint Approval
    # ============================================================================
    feedback = input("\n👤 Type 'Approved' to proceed, or provide revision requests: ")
    
    if feedback.lower() == "approved":
        # Blueprint accepted - move to code generation
        state["status"] = "approved"
        state["user_feedback"] = None  # Clear any previous feedback
        print("✅ Blueprint Approved!")
        break  # Exit the revision loop
    else:
        # User wants changes - loop back to Architect
        state["status"] = "revision_requested"
        state["user_feedback"] = feedback
        print(f"\n🔄 Revision Requested: {feedback}")
        print("📐 Sending back to Architect for refinement...")
        
        # Re-run Architect with feedback incorporated
        update = architect_node(state)
        state.update(update)
        iteration += 1

# ============================================================================
# PHASE 3: CODE GENERATION
# ============================================================================
print("\n" + "=" * 70)
print("PHASE 3: CODE GENERATION")
print("=" * 70)
print("👷 Builder is generating your agent system...")

# Run Builder agent to generate final code
update = builder_node(state)
state.update(update)

# ============================================================================
# FINAL OUTPUT - Display and Save Generated Code
# ============================================================================
print("\n" + "=" * 70)
print("🎉 AGENT FACTORY COMPLETE!")
print("=" * 70)
print("\n📄 GENERATED CODE:")
print("-" * 70)
print(state['final_code'])
print("-" * 70)

# Save to file for immediate use
output_filename = "generated_langchain_agent.py"
with open(output_filename, "w") as f:
    f.write(state['final_code'])

print(f"\n💾 Code saved to: {output_filename}")
print("✅ Your custom agent system is ready to use!")
print("\n🚀 Next Steps:")
print("   1. Review the generated code")
print("   2. Install required dependencies (pip install langchain langgraph)")
print("   3. Run: python generated_langchain_agent.py")
print("=" * 70)

---

## 🎓 Learning Outcomes & Next Steps

### 🧠 What You Learned
- ✅ **LangChain Integration**: Using ChatOpenAI for LLM orchestration
- ✅ **LangGraph Workflows**: Building StateGraph with nodes and edges
- ✅ **Human-in-the-Loop Design**: Interactive AI systems with checkpoints
- ✅ **Multi-Agent Architectures**: Consultant → Architect → Builder pattern
- ✅ **State Management**: TypedDict for shared memory across agents

### 🔄 Comparison: LangChain vs Google ADK

| Aspect | **LangChain Version** | **Google ADK Version** |
|--------|----------------------|------------------------|
| **Graph Definition** | Explicit StateGraph | Implicit while-loop |
| **State Type** | TypedDict | Native dict/class |
| **Flow Control** | END + manual invoke | Loop conditions |
| **LLM Provider** | OpenAI (swappable) | Google Gemini (fixed) |
| **Tool Ecosystem** | LangChain tools | Google ADK tools |

### 🚀 Extending This System

#### 1. **Add More Agents**
```python
workflow.add_node("tester", tester_node)  # Validate generated code
workflow.add_node("documenter", doc_node)  # Auto-generate docs
```

#### 2. **Automate Approvals**
Replace human checkpoints with automated validation:
```python
def auto_approve_blueprint(state):
    # Parse JSON, validate structure
    try:
        blueprint = json.loads(state['current_blueprint'])
        if validate_blueprint(blueprint):
            return "approved"
    except:
        return "revision_requested"
```

#### 3. **Add Tool Integration**
Give agents real capabilities:
```python
from langchain.agents import load_tools
tools = load_tools(["serpapi", "python_repl"])
```

#### 4. **Persistent Storage**
Save blueprints and code to database:
```python
from langchain.storage import InMemoryStore
storage = InMemoryStore()
storage.mset([(f"blueprint_{timestamp}", state['current_blueprint'])])
```

### 📚 Resources
- **LangChain Docs**: https://python.langchain.com/
- **LangGraph Docs**: https://langchain-ai.github.io/langgraph/
- **OpenAI API**: https://platform.openai.com/docs

### 🎯 Real-World Applications
This pattern works for:
- **Software Architecture**: Design microservices, APIs, databases
- **Content Creation**: Articles, videos, marketing campaigns
- **Data Pipelines**: ETL workflows, analytics systems
- **DevOps Automation**: Infrastructure as Code, CI/CD pipelines

---

### 🙏 Credits
Built with:
- **LangChain** for LLM orchestration
- **LangGraph** for workflow management
- **OpenAI GPT-4o** for reasoning
- **Python-dotenv** for configuration management

---